Abdulrahman Emad 18101010
Mohamed Yasser Anwar 18100585
Mohamed Ashraf 18100765

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
import matplotlib.pyplot as plt
import datetime
from sklearn import metrics
import seaborn as sns
import sklearn.ensemble as ske
import gc
from sklearn.linear_model import LinearRegression,SGDRegressor,ElasticNet,Ridge
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
DATA_PATH = "../input/ashrae-energy-prediction/"
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

* **** Loading Data

In [ ]:
Weather_Train = pd.read_csv(DATA_PATH + 'weather_train.csv')
Weather_Test= weather_df = pd.read_csv(DATA_PATH + 'weather_test.csv')
Test= pd.read_csv(DATA_PATH +'test.csv')
Train= pd.read_csv(DATA_PATH +'train.csv')
Building= pd.read_csv(DATA_PATH +'building_metadata.csv')

* **** About the data

In [ ]:
#Function that prints Shape of the data, Columns and their datatype, Head of the data, Description of the data, number of nulls in the data
# and Year of the data and The Percentage of missing values in the dataset
def Data_info(filename):
    path = "../input/ashrae-energy-prediction"
    df = pd.read_csv('{0}/{1}'.format(path,filename))
    print("~~~~~~Shape of the data~~~~~~ : ",df.shape)
    print("~~~~~~Columns and their datatype~~~~~~ : ")
    print(df.info())
    print("~~~~~~Quick Look at the data~~~~~~ : ")
    print(df.head())
    print("~~~~~~Description of the data~~~~~~ : ")
    print(df.describe())
    print("~~~~~~number of nulls in the data~~~~~~ : ")
    print(df.isna().sum())
    if 'timestamp' in df.columns: 
        df['timestamp'] = pd.to_datetime(df['timestamp'],format = "%Y-%m-%d %H:%M:%S")
        print("~~~~~~Year of the data~~~~~~ :")
        print(df.timestamp.dt.year.unique())
    print("~~~~~~Percentage of missing values in the dataset~~~~~~")
    print(df.isna().sum()/len(df)*100)
    return df

In [ ]:
#Using ("Data_info") function to get 'train.csv' info
train = Data_info('train.csv')

In [ ]:
#Using ("Data_info") function to get 'test.csv' info
test = Data_info('test.csv')

In [ ]:
#Using ("Data_info") function to get 'weather_train.csv' info
wtrain = Data_info('weather_train.csv')

In [ ]:
#Using ("Data_info") function to get 'weather_test.csv' info
wtest = Data_info('weather_test.csv')

In [ ]:
#Using ("Data_info") function to get 'building_metadata.csv' info
building = Data_info('building_metadata.csv')

In [ ]:
#Function That reduces the used memory
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
#using reduce_mem_usage function to reduce memory usage
Train = reduce_mem_usage(Train)
Test = reduce_mem_usage(Test)
Weather_Train = reduce_mem_usage(Weather_Train)
Weather_Test = reduce_mem_usage(Weather_Test)
Building = reduce_mem_usage(Building)

* **EDA**

In [ ]:
#Merging tables
results = Building.merge(Train,left_on='building_id',right_on='building_id',how='left')
data = results.merge(Weather_Train,left_on=['site_id','timestamp'],right_on=['site_id','timestamp'],how='left')
data.columns

In [ ]:
#Function that sets up the hist plot and its dimension
def plot_hist(df,var_name):
    plt.figure(figsize=(17,8))
    plt.hist(df[var_name],bins = 50)
    plt.title(f"Histogram - {var_name}")
    plt.show()

In [ ]:
#Hist plot for 'year_built'
plot_hist(Building,'year_built')

In [ ]:
#Hist plot for 'floor_count'
plot_hist(Building,'floor_count')

In [ ]:
#Hist plot for 'cloud_coverage'
plot_hist(data,'cloud_coverage')

In [ ]:
#Hist plot for 'wind_speed'
plot_hist(data,'wind_speed')

In [ ]:
#Hist plot for 'dew_temperature','air_temperature' & 'wind_speed'
for var in ['dew_temperature','air_temperature','wind_speed']:
    plot_hist(Weather_Train,var)

In [ ]:
#Scatter plot between Air Temperature and Meter Reading
sns.scatterplot(x='air_temperature', y='meter_reading', data=data)

In [ ]:
#Scatter plot between Cloud Coverage and Meter Reading
#sns.scatterplot(x='cloud_coverage', y='meter_reading', hue='year_built',data=data)
#it works but it takes a lot of time to run

In [ ]:
#Boxplots for Meter reading for Floorcounts
fig, axes = plt.subplots(1, 1, figsize=(14, 6))
sns.boxplot(x='floor_count', y='meter_reading', data=data, showfliers=False);

In [ ]:
#Boxplots for 'site_id' and 'dew_temperature'
fig, axes = plt.subplots(1, 1, figsize=(14, 6))
sns.boxplot(x='site_id', y='dew_temperature', data=data, showfliers=False);

In [ ]:
#Heatmap for the correlation between the columns and themselvs
corrmat=data.corr()
fig,ax=plt.subplots(figsize=(12,10))
sns.heatmap(corrmat,annot=True,annot_kws={'size': 12})

* **Features Engineering**

In [ ]:
#dropping unnecessary columns
data = data.drop(columns=['year_built', 'floor_count', 'wind_direction', 'dew_temperature'])

In [ ]:
#fixing timestamp and taking only the day and the month and then dropping timestamp column
data["timestamp"] = pd.to_datetime(data["timestamp"])
data["day"]= data["timestamp"].dt.day
data["month"]= data["timestamp"].dt.month
data= data.drop("timestamp", axis = 1)

In [ ]:
#Printing the "Percentage of missing values" to see what is going to be dropped and what is going to be filled
print("Percentage of missing values in the data dataset")
data.isna().sum()/len(data)*100

In [ ]:
# Since there is a problem in printing the columns, instead printing head of the data besides the columns inside ("data")
data.head()

In [ ]:
#setting a list of integer ranging from 0 to length of data as index
data = data.reset_index()

In [ ]:
#Change data type to float 32 for filling NA value before transforming them into int for smooth modeling processing
data['wind_speed'] = data['wind_speed'].astype('float32')
data['air_temperature'] = data['air_temperature'].astype('float32')
data['precip_depth_1_hr'] = data['precip_depth_1_hr'].astype('float32')
data['cloud_coverage'] = data['cloud_coverage'].astype('float32')

In [ ]:
#Filling Null Values
data['precip_depth_1_hr'].fillna(data['precip_depth_1_hr'].mean(), inplace = True)
data['cloud_coverage'].fillna(data['cloud_coverage'].mean(), inplace = True)
data['wind_speed'].fillna(data['wind_speed'].mean(), inplace=True)
data['air_temperature'].fillna(data['air_temperature'].mean(), inplace=True)

# Printing the sum of nulls inside the columns
data.isnull().sum()

In [ ]:
#printing the shape of the "results"
results.shape

In [ ]:
#printing the columns of the "results"
results.columns

In [ ]:
#printing the shape of the "data"
data.shape

In [ ]:
#printing the shape of the "data"
data.columns

* **Linear Model**

In [ ]:
# Here column 'primaty_use' was treated by get_dummies function and get_dummies is used for data manipulation
data_linearR = pd.get_dummies(data, columns=['primary_use'])

In [ ]:
#printing the columns of "data_linearR"
data_linearR.columns

In [ ]:
#Using the important features 
X =data_linearR[['building_id', 'meter', 'air_temperature', 'wind_speed', 'precip_depth_1_hr', 'cloud_coverage',
       'square_feet', 'primary_use_Education', 'primary_use_Entertainment/public assembly',
       'primary_use_Food sales and service', 'primary_use_Healthcare',
       'primary_use_Lodging/residential',
       'primary_use_Manufacturing/industrial', 'primary_use_Office',
       'primary_use_Other', 'primary_use_Parking',
       'primary_use_Public services', 'primary_use_Religious worship',
       'primary_use_Retail', 'primary_use_Services',
       'primary_use_Technology/science', 'primary_use_Utility',
       'primary_use_Warehouse/storage', 'month', 'day']]

# Create target variable
y = data_linearR['meter_reading']

# Train, test, split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .20, random_state= 0)

In [ ]:
# Create linear regression object
regressor = LinearRegression()

# Fit model to training data
regressor.fit(X_train,y_train)

In [ ]:
# Predicting test set results
y_pred = regressor.predict(X_test)

In [ ]:
#Printing R^2 and the accuracy of the linear model
print('Accuracy %d', regressor.score(X_test, y_test))
print('R^2 =', metrics.explained_variance_score(y_test,y_pred))
regressor.score(X_train,y_train)